# Анализ Датасета IMDB 

#### Задача: ответить правильно на все вопросы по анализу датасета, используя библиотеку pandas, соблюсти стандарт pep-8, написать красивый код. 

## Инициализация

In [1694]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from itertools import combinations


In [1695]:
data = pd.read_csv('movie_bd_v5.csv')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
1740,tt0498353,10200000,35619521,Hostel: Part II,Lauren German|Bijou Phillips|Heather Matarazzo...,Eli Roth,Americans...they have no imagination.,"Following a geographical tour of Slovakia, thr...",93,Horror,Lions Gate Films|Next Entertainment|Screen Gems,6/8/2007,5.6,2007
1804,tt0120755,125000000,546388105,Mission: Impossible II,Tom Cruise|Dougray Scott|Thandie Newton|Ving R...,John Woo,Expect the impossible again,With computer genius Luther Stickell at his si...,123,Adventure|Action|Thriller,Paramount Pictures|Cruise/Wagner Productions|M...,5/23/2000,5.8,2000
943,tt0307479,47000000,30002758,Solaris,George Clooney|Natascha McElhone|Viola Davis|J...,Steven Soderbergh,How far will you go for a second chance?,Upon arrival at the space station orbiting an ...,99,Drama|Science Fiction|Mystery|Romance,Twentieth Century Fox Film Corporation|Lightst...,11/27/2002,5.8,2002
1827,tt0223897,40000000,55707411,Pay It Forward,Kevin Spacey|Helen Hunt|Haley Joel Osment|Jay ...,Mimi Leder,"When someone does you a big favor, don't pay i...","Like some other kids, 12-year-old Trevor McKin...",122,Drama|Romance,Bel Air Entertainment|Warner Bros.,10/12/2000,6.8,2000
680,tt0443701,30000000,68369434,The X Files: I Want to Believe,David Duchovny|Gillian Anderson|Amanda Peet|Bi...,Chris Carter,"To find the truth, you must believe.",Six years after the events of The X-Files seri...,104,Drama|Mystery|Science Fiction|Thriller,Twentieth Century Fox Film Corporation|Ten Thi...,7/24/2008,5.4,2008


In [1696]:
# Словарь со всеми ответами к вопросам
answers = {} 

## Предобработка Данных

In [1697]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   imdb_id               1889 non-null   object 
 1   budget                1889 non-null   int64  
 2   revenue               1889 non-null   int64  
 3   original_title        1889 non-null   object 
 4   cast                  1889 non-null   object 
 5   director              1889 non-null   object 
 6   tagline               1889 non-null   object 
 7   overview              1889 non-null   object 
 8   runtime               1889 non-null   int64  
 9   genres                1889 non-null   object 
 10  production_companies  1889 non-null   object 
 11  release_date          1889 non-null   object 
 12  vote_average          1889 non-null   float64
 13  release_year          1889 non-null   int64  
dtypes: float64(1), int64(4), object(9)
memory usage: 206.7+ KB


#### Проверка на  null значения

In [1698]:
data.isnull().values.any()

False

#### Проверяем все ли столбцы находятся в подходящем для них дата формате 

In [1699]:
object_data = data.select_dtypes(include = 'object').columns
object_data

Index(['imdb_id', 'original_title', 'cast', 'director', 'tagline', 'overview',
       'genres', 'production_companies', 'release_date'],
      dtype='object')

In [1700]:
for ob in object_data:
    print(type(ob),'\t', ob)

<class 'str'> 	 imdb_id
<class 'str'> 	 original_title
<class 'str'> 	 cast
<class 'str'> 	 director
<class 'str'> 	 tagline
<class 'str'> 	 overview
<class 'str'> 	 genres
<class 'str'> 	 production_companies
<class 'str'> 	 release_date


#### Меняем формат столбцов с неправильным форматом на правильный 

In [1701]:
data['release_date'] = pd.to_datetime(data['release_date'])
type(data['release_date'][0])

pandas._libs.tslibs.timestamps.Timestamp

### Вспомогательные функции 

#### Отформатировать  месяцы 

In [1702]:
def format_month_df(months):
    months.rename(columns = {'release_date':'count'}, inplace = True)
    months.index.names = ['release_month']
    month_names = ['January', 'Feburary', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'Ocotber', 'November', 'December']
    months.index = map(lambda x: month_names[x-1], months.index)
    
    return months

#### Добавить profit data столбец 

In [1703]:
data['profit'] = data['revenue'] - data['budget']
data['profit']

0       1363528810
1        228436354
2        185238201
3       1868178225
4       1316249360
           ...    
1884      82299717
1885      -8444012
1886     -40865180
1887      -3962091
1888      -9782502
Name: profit, Length: 1889, dtype: int64

#### Конверитировать list в  pd.Series 

In [1704]:
def list_to_Series(x): 
    idx, values = zip(*x)
    return pd.Series(values, idx)

#### Преобразовать строки столбца в  lists

In [1705]:
def to_one_str(column):
    return column.str.cat(sep = '|').split('|')

#### Найти самое частое слово в строке

In [1706]:
def most_common(column):
    return Counter(to_one_str(column)).most_common()

#### Получить отсортированные студии по полученному критерию

In [1707]:
def get_studios(col):
    studios = df.groupby(['production_companies']).median(col)
    studios = studios.sort_values(by = col, ascending = False)

    return studios

#### Посчитать длину строк столбца

In [1708]:
def calc_len(column):
    return list(map(lambda x: len(x), column))

#### Занести ответ в словарь

In [1709]:
# id_name = True, если ответ формата "имя_фильма (id_фильма)", иначе False
def answer(answer, question, id_name = False):
    if id_name:
        answers[question] = answer['original_title'].item() + ' (' + answer['imdb_id'].item() + ')'
    else: 
        answers[question] = answer
answers

{}

# Анализ Данных

# 1. У какого фильма из списка самый большой бюджет?

In [1710]:
max_budget = data[data['budget'] == data['budget'].max()]
answer(max_budget, 1, True)

answers[1]

'Pirates of the Caribbean: On Stranger Tides (tt1298650)'

# 2. Какой из фильмов самый длительный (в минутах)?

In [1711]:
longest = data[data['runtime'] == data['runtime'].max()]
answer(longest, 2, True)

answers[2]

'Gods and Generals (tt0279111)'

# 3. Какой из фильмов самый короткий (в минутах)?





In [1712]:
shortest = data[data['runtime'] == data['runtime'].min()]
answer(shortest, 3, True)

answers[3]

'Winnie the Pooh (tt1449283)'

# 4. Какова средняя длительность фильмов?


In [1713]:
avg_runtime = data['runtime'].mean()
answer(avg_runtime, 4)

answers[4]

109.6585494970884

# 5. Каково медианное значение длительности фильмов? 

In [1714]:
meadian_runtime = data['runtime'].median()
answer(meadian_runtime, 5)

answers[5]

107.0

# 6. Какой самый прибыльный фильм?
Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [1715]:
most_profit = data[data['profit']==data['profit'].max()]
answer(most_profit, 6, True)

answers[6]

'Avatar (tt0499549)'

# 7. Какой фильм самый убыточный? 

In [1716]:
least_profit = data.query('profit == profit.min()')
answer(least_profit, 7, True)

answers[7]

'The Lone Ranger (tt1210819)'

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [1717]:
pos_profit = data[data['profit']>0].count()['imdb_id']
answer(pos_profit, 8)

answers[8]

1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [1718]:
movies_2008 = data.query('release_year == 2008')
highest_gross =  movies_2008.query('revenue == revenue.max()')

answer(highest_gross, 9, True)
answers[9]

'The Dark Knight (tt0468569)'

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?

In [1719]:
period = data[data['release_year'].between(2012,2014)]

most_loss = period.query('profit == profit.min()')
answer(most_loss, 10, True)

answers[10]

'The Lone Ranger (tt1210819)'

# 11. Какого жанра фильмов больше всего?

In [1742]:
df = data[['genres', 'original_title']].copy()
df['genres'] = split(data['genres'])
df = df.explode('genres')

genres = df.groupby(['genres']).count().sort_values(by = 'original_title', ascending=False)

answer(genres.iloc[0].name, 11)
answers[11]

'Drama'

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [1721]:
profitable = data[data.profit > 0]
quantity = most_common(profitable['genres'])

display(quantity)
answer(quantity[0][0], 12)

[('Drama', 560),
 ('Comedy', 551),
 ('Thriller', 446),
 ('Action', 444),
 ('Adventure', 337),
 ('Romance', 242),
 ('Crime', 231),
 ('Family', 226),
 ('Science Fiction', 195),
 ('Fantasy', 188),
 ('Horror', 150),
 ('Animation', 120),
 ('Mystery', 119),
 ('Music', 47),
 ('History', 46),
 ('War', 41),
 ('Western', 12),
 ('Documentary', 7)]

# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [1722]:
directors = data.groupby(['director'])
movie_shark = directors['revenue'].sum().sort_values(ascending = False)

answer(movie_shark.index[0], 13)
movie_shark.head(5)

director
Peter Jackson        6490593685
Christopher Nolan    4167548502
David Yates          4154295625
Michael Bay          3886938960
J.J. Abrams          3579169916
Name: revenue, dtype: int64

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [1723]:
action = data[data['genres'].str.contains('Action')][['director','genres']]
directs_most = list_to_Series(most_common(action['director']))

display(directs_most.head())

answer(directs_most.index[0],14)

Robert Rodriguez      9
Michael Bay           7
Paul W.S. Anderson    7
Antoine Fuqua         6
Ridley Scott          6
dtype: int64

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [1724]:
movies_12 = data[data['release_year']==2012][['original_title', 'cast', 'revenue']]

top_movies = movies_12.query('revenue > revenue.mean()').sort_values(['revenue'],ascending=False)
pop_actors = list_to_Series(most_common(top_movies['cast']))

display(pop_actors.head())

answer(pop_actors.index[0], 15)

Chris Hemsworth        2
Ralph Fiennes          2
Anne Hathaway          2
Denis Leary            2
Seann William Scott    2
dtype: int64

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [1725]:
high_budget = data.query('budget > budget.mean()')[['original_title','cast','budget']]
most_pop_actor = list_to_Series(most_common(high_budget['cast']))

answer(most_pop_actor.index[0], 16)
most_pop_actor.head(3)


Matt Damon        18
Adam Sandler      17
Angelina Jolie    16
dtype: int64

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [1726]:
cages_movies = data[data['cast'].str.find('Nicolas Cage')>=0][['original_title', 'cast', 'genres']]

# Самый частый жанр, в котором снимается Кейдж
cages_pop_genre = list_to_Series(most_common(cages_movies['genres']))

answer(cages_pop_genre.index[0], 17)
cages_pop_genre.head()

Action      17
Thriller    15
Drama       12
Crime       10
Fantasy      8
dtype: int64

# 18. Самый убыточный фильм от Paramount Pictures

In [1727]:
paramount = data[data['production_companies'].str.find('Paramount Pictures')>=0]
paramount = paramount[['imdb_id','original_title','profit']]

unsuccessful = paramount.query('profit == profit.min()')

answer(unsuccessful, 18, True)
answers[18]

'K-19: The Widowmaker (tt0267626)'

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [1728]:
revenue = data.groupby('release_year')['revenue'].sum().sort_values(ascending=False)
display(revenue.head())

answer(revenue.index[0], 19)
answers[19]

release_year
2015    25449202382
2014    23405862953
2013    23213799791
2012    23079001687
2011    22676791872
Name: revenue, dtype: int64

2015

# 20. Какой самый прибыльный год для студии Warner Bros?

In [1729]:
WB_movies = data[data['production_companies'].str.find('Warner Bros')>0][['release_year','profit']]

most_profit_year = WB_movies.groupby('release_year').sum().sort_values(['profit'],ascending=False)

display(most_profit_year.head(3))

answer(most_profit_year.index[0], 20)
answers[20]

,profit
release_year,
2014,2297979392
2008,2134595031
2007,1928180689


2014

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [1730]:
months = data['release_date'].apply(lambda x: x.month)
months = format_month_df(months.groupby(months).count().to_frame())

highest_num = months.sort_values(by = ['count'], ascending = False).index[0]

answer(highest_num, 21)
answers[21]

'September'

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [1731]:
values = months[5:8]['count'].values
summer_mov = sum(values)

answer(summer_mov, 22)
answers[22]

450

# 23. Для какого режиссера зима – самое продуктивное время года? 

#### Общая часть для обоих решений: инициализация 

In [1732]:
df = data[['release_date','director']].copy()
df['release_date'] = df['release_date'].apply(lambda x: x.month)

winter = [12,1,2]
df = df[df['release_date'].isin(winter)]

#### Вариант 1: использование counter

In [1733]:
winter_directors = most_common(df['director'])

answer(winter_directors[0][0], 23)
answers[23]

'Peter Jackson'

#### Вариант 2: использование explode

In [1734]:
df['director'] = split(df['director'])
df = df.explode('director')

winter_directors = df.groupby(['director']).count().sort_values(by= 'release_date', ascending = False)

answer(winter_directors.iloc[0].name, 23)
answers[23]

'Peter Jackson'

# Вопросы 24 and 25

#### Общая часть для кода к вопросам 24 и 25: инициализация 

In [1735]:
df = data['production_companies'].copy().to_frame()

# Добавляем 2 столбца, отражающие длину описаний и названий фильмов
df.insert(1, "title_len", calc_len(data['original_title']), True)
df.insert(2, "overview_len", calc_len(data['overview']), True)

df['production_companies'] = split(df['production_companies'])
df = df.explode('production_companies')

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [1736]:
studios = get_studios('title_len')

display(studios)

answer(studios.iloc[0].name, 24) 

,title_len,overview_len
production_companies,,
Four By Two Productions,83.0,298.0
"Jim Henson Company, The",59.0,304.0
Dos Corazones,47.0,150.0
Museum Canada Productions,46.0,302.0
Polsky Films,46.0,387.0
...,...,...
Everest Entertainment,3.0,124.0
Berlanti Productions,3.0,413.0
Ixtlan Productions,2.0,340.0


# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [1737]:
studios = get_studios('overview_len')

display(studios)

answer(studios.iloc[0].name, 25) 

,title_len,overview_len
production_companies,,
Midnight Picture Show,23.0,1000.0
Room 9 Entertainment,21.0,964.0
Brookwell-McNamara Entertainment,11.0,936.0
Lions Gate Family Entertainment,15.0,909.0
Crest Animation Productions,15.0,909.0
...,...,...
Projection Pictures,16.0,74.0
London Boulevard,16.0,74.0
Phantom Four,10.0,72.0


# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 

In [1738]:
best_movies = data[['imdb_id', 'original_title', 'vote_average']].sort_values('vote_average', ascending = False)
quantity = len(best_movies)
one_percent = round(quantity * 0.01)

display(best_movies[:one_percent])

answer(list(best_movies[:one_percent]['original_title'].values), 26)

,imdb_id,original_title,vote_average
599,tt0468569,The Dark Knight,8.1
118,tt0816692,Interstellar,8.0
125,tt2084970,The Imitation Game,8.0
9,tt2096673,Inside Out,8.0
34,tt3170832,Room,8.0
1183,tt0993846,The Wolf of Wall Street,7.9
128,tt2267998,Gone Girl,7.9
1191,tt2024544,12 Years a Slave,7.9
119,tt2015381,Guardians of the Galaxy,7.9
1081,tt0167260,The Lord of the Rings: The Return of the King,7.9


# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [1739]:
actors = data[['original_title', 'cast']].copy()

actors['cast'] = split(actors['cast'])

# Находим все возможные комбинации пар актеров, снявшихся в одном фильме
actors['combinations'] = actors['cast'].apply(lambda cast: list(combinations(cast,2)))
actors = actors.explode('combinations')

# Группирем и считем кол-во раз актеры снимались вместе
often_together = actors.groupby('combinations').count().sort_values('cast',ascending= False)
the_pairs = list(often_together.query('cast == cast.max()').index)

answer(the_pairs, 27)
answers[27]

[('Daniel Radcliffe', 'Rupert Grint'), ('Daniel Radcliffe', 'Emma Watson')]

# Вывод

In [1740]:
# Ответы к вопросам
answers

{1: 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 2: 'Gods and Generals (tt0279111)',
 3: 'Winnie the Pooh (tt1449283)',
 4: 109.6585494970884,
 5: 107.0,
 6: 'Avatar (tt0499549)',
 7: 'The Lone Ranger (tt1210819)',
 8: 1478,
 9: 'The Dark Knight (tt0468569)',
 10: 'The Lone Ranger (tt1210819)',
 11: original_title    782
 Name: Drama, dtype: int64,
 12: 'Drama',
 13: 'Peter Jackson',
 14: 'Robert Rodriguez',
 15: 'Chris Hemsworth',
 16: 'Matt Damon',
 17: 'Action',
 18: 'K-19: The Widowmaker (tt0267626)',
 19: 2015,
 20: 2014,
 21: 'September',
 22: 450,
 23: 'Peter Jackson',
 24: 'Four By Two Productions',
 25: 'Midnight Picture Show',
 26: ['The Dark Knight',
  'Interstellar',
  'The Imitation Game',
  'Inside Out',
  'Room',
  'The Wolf of Wall Street',
  'Gone Girl',
  '12 Years a Slave',
  'Guardians of the Galaxy',
  'The Lord of the Rings: The Return of the King',
  'Memento',
  'Inception',
  'The Pianist',
  'The Grand Budapest Hotel',
  'Her',
  'Spotlight',
  

In [1741]:
len(answers)

27